## Seperable Conv2D vs. Con2D
即使模型設置都一模一樣，但是參數量明顯減少非常多！

| Layers                        | Parameters for Conv2D    | Parameters for SeparableConv2D       |
|-------------------------------|--------------------------|--------------------------------------|
| conv2d_1 (Conv2D)             | (3x3x32)x3 + 32 = 896    | (3x3x1)x3 + (1x1x3)x32 + 32 = 155    |
| max_pooling2d_1 (MaxPooling2) |                          |                                      |
| conv2d_2 (Conv2D)             | (3x3x64)x32 + 64 = 18496 | (3x3x1)x32 + (1x1x32)x64 + 64 = 2400 |
| max_pooling2d_2 (MaxPooling2) |                          |                                      |
| conv2d_3 (Conv2D)             | (3x3x64)x64 + 64 = 36928 | (3x3x1)x64 + (1x1x64)x64 + 64 = 4736 |
| flatten_1 (Flatten)           | (3x3x64) = 576           | (3x3x64) = 576                       |
| dense_1 (Dense)               | (576+1)x256 = 147712     | (576+1)x256 = 147712                 |
| dense_2 (Dense)               | (256+1)x10 = 2570        | (256+1)x10 = 2570                    |

In [0]:
from keras import layers
from keras import models
from keras import backend as K

In [0]:
# Conv2D
K.clear_session()
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)              

In [0]:
# SeparableConv2D
K.clear_session()
model = models.Sequential()
model.add(layers.SeparableConv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_1 (Separabl (None, 26, 26, 32)        155       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 11, 11, 64)        2400      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 3, 3, 64)          4736      
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)              

## 作業

請閱讀 Keras 官方範例 [mnist_cnn.py](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py)  

並回答下列問題。僅有 70 行程式碼，請確保每一行的程式碼你都能夠理解目的

1. 是否有對資料做標準化 (normalization)? 如果有，在哪幾行?

   37, 38 行
  ```python
  x_train /= 255
  x_test /= 255
  ```

2. 使用的優化器 Optimizer 為何?

  Adadelta
  ```python
  model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
  ```
3. 模型總共疊了幾層卷積層?

  2 層
  ```python
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))
  ```
4. 模型的參數量是多少?

  1,200,458